# Neural Networks with Keras

513/513 - 0s - loss: 1.7734 - acc: 0.2710

Loss: 1.7734287705337792, Accuracy: 0.2709551751613617

uses minmaxscaler

ref: 21-Machine-Learning/3/Activities/02-Evr_First_Neural_Network/Solved/First_Neural_Network.ipynb#Model-Summary

In [1]:
# Dependencies
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn import tree
import os
from sklearn.datasets import make_classification

Import csv that has been optimized for 10 freatures

In [2]:
# import processed data

path = "data/"
file = "Best_disposition_data.csv"
path_file = path + file

df = pd.read_csv(path_file)
df = df.drop("Unnamed: 0", axis=1)
df

,DATE_year,DATE_month,DATE_week,DATE_day,Hour,Minute,Temp(K),Humidity%,Pressure(atm),wind,Description,disposition
0,2012,10,40,4,20,0,293.31,56.0,1017.0,7.0,0,0
1,2012,10,40,4,23,0,291.82,68.0,1017.0,3.0,13,2
2,2012,10,40,5,0,1,289.74,63.0,1018.0,3.0,13,0
3,2012,10,41,10,14,0,281.87,66.0,1020.0,2.0,2,2
4,2012,10,42,15,12,30,292.15,82.0,1009.0,8.0,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2047,2017,8,32,11,13,0,293.42,88.0,1017.0,2.0,14,4
2048,2017,8,33,20,20,29,304.17,54.0,1018.0,1.0,14,2
2049,2017,9,39,27,17,30,297.76,73.0,1018.0,5.0,0,2
2050,2017,10,41,13,16,30,289.81,67.0,1023.0,2.0,13,2


# Data Preprocessing
It is really important to scale our data before using multilayer perceptron models.

Without scaling, it is often difficult for the training cycle to converge

In [3]:
X = df.drop("disposition", axis=1)
y = df["disposition"]
print(X.shape, y.shape)

(2052, 11) (2052,)


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

C:\Users\phoog\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\phoog\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\phoog\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\phoog\anaconda3\envs\PythonData\lib\site-packages

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [6]:
# MinMaxScaler is used

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [7]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [8]:
#n= 0
#for label, original_class in zip(encoded_y, y):
#    print('Original Class: ' + str(original_class))
#    print('Encoded Label: ' + str(label))    
#    n=n+1
#    print(n)
#    print('-' * 12)

In [9]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

## Creating our Model

Decide what kind of model to apply to our data. 

    For numerical data, we use a regressor model. 

    For categorical data, we use a classifier model. 

In this example, we will use a classifier to build the following network:

## Defining our Model Architecture (the layers)

Create a sequential model

In [10]:
from tensorflow.keras.models import Sequential

model = Sequential()

In [11]:
from tensorflow.keras.layers import Dense
from tensorflow.python.ops.init_ops import VarianceScaling


number_inputs = 11
number_hidden_nodes = 12
model.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_inputs))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [12]:
number_classes = 6
model.add(Dense(units=number_classes, activation='softmax'))

## Model Summary

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                144       
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 78        
Total params: 222
Trainable params: 222
Non-trainable params: 0
_________________________________________________________________


## Compile the Model

Now that we have our model architecture defined, we must compile the model using a loss function and optimizer. We can also specify additional training metrics such as accuracy.

In [14]:
# Use categorical crossentropy for categorical data and mean squared error for regression
# Hint: your output layer in this example is using software for logistic regression (categorical)
# If your output layer activation was `linear` then you may want to use `mse` for loss
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## Training the Model
Finally, we train our model using our training data
Training consists of updating our weights using our optimizer and loss function. In this example, we choose 1000 iterations (loops) of training that are called epochs.

We also choose to shuffle our training data and increase the detail printed out during each training cycle.

In [15]:
# Fit (train) the model
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=1000,
    shuffle=True,
    verbose=2
)

Epoch 1/1000
1539/1539 - 0s - loss: 1.7774 - acc: 0.2359
Epoch 2/1000
1539/1539 - 0s - loss: 1.7682 - acc: 0.2424
Epoch 3/1000
1539/1539 - 0s - loss: 1.7632 - acc: 0.2482
Epoch 4/1000
1539/1539 - 0s - loss: 1.7604 - acc: 0.2521
Epoch 5/1000
1539/1539 - 0s - loss: 1.7576 - acc: 0.2521
Epoch 6/1000
1539/1539 - 0s - loss: 1.7550 - acc: 0.2495
Epoch 7/1000
1539/1539 - 0s - loss: 1.7530 - acc: 0.2508
Epoch 8/1000
1539/1539 - 0s - loss: 1.7514 - acc: 0.2528
Epoch 9/1000
1539/1539 - 0s - loss: 1.7503 - acc: 0.2554
Epoch 10/1000
1539/1539 - 0s - loss: 1.7473 - acc: 0.2586
Epoch 11/1000
1539/1539 - 0s - loss: 1.7460 - acc: 0.2586
Epoch 12/1000
1539/1539 - 0s - loss: 1.7442 - acc: 0.2625
Epoch 13/1000
1539/1539 - 0s - loss: 1.7434 - acc: 0.2736
Epoch 14/1000
1539/1539 - 0s - loss: 1.7417 - acc: 0.2762
Epoch 15/1000
1539/1539 - 0s - loss: 1.7403 - acc: 0.2755
Epoch 16/1000
1539/1539 - 0s - loss: 1.7391 - acc: 0.2801
Epoch 17/1000
1539/1539 - 0s - loss: 1.7377 - acc: 0.2833
Epoch 18/1000
1539/1539

Epoch 142/1000
1539/1539 - 0s - loss: 1.7035 - acc: 0.3034
Epoch 143/1000
1539/1539 - 0s - loss: 1.7031 - acc: 0.2956
Epoch 144/1000
1539/1539 - 0s - loss: 1.7029 - acc: 0.2982
Epoch 145/1000
1539/1539 - 0s - loss: 1.7024 - acc: 0.2989
Epoch 146/1000
1539/1539 - 0s - loss: 1.7025 - acc: 0.2976
Epoch 147/1000
1539/1539 - 0s - loss: 1.7024 - acc: 0.2976
Epoch 148/1000
1539/1539 - 0s - loss: 1.7024 - acc: 0.2982
Epoch 149/1000
1539/1539 - 0s - loss: 1.7021 - acc: 0.2963
Epoch 150/1000
1539/1539 - 0s - loss: 1.7023 - acc: 0.2976
Epoch 151/1000
1539/1539 - 0s - loss: 1.7022 - acc: 0.2976
Epoch 152/1000
1539/1539 - 0s - loss: 1.7013 - acc: 0.2989
Epoch 153/1000
1539/1539 - 0s - loss: 1.7015 - acc: 0.3021
Epoch 154/1000
1539/1539 - 0s - loss: 1.7012 - acc: 0.2963
Epoch 155/1000
1539/1539 - 0s - loss: 1.7013 - acc: 0.2956
Epoch 156/1000
1539/1539 - 0s - loss: 1.7020 - acc: 0.3002
Epoch 157/1000
1539/1539 - 0s - loss: 1.7011 - acc: 0.3002
Epoch 158/1000
1539/1539 - 0s - loss: 1.7005 - acc: 0.29

Epoch 281/1000
1539/1539 - 0s - loss: 1.6874 - acc: 0.3067
Epoch 282/1000
1539/1539 - 0s - loss: 1.6878 - acc: 0.3106
Epoch 283/1000
1539/1539 - 0s - loss: 1.6872 - acc: 0.3093
Epoch 284/1000
1539/1539 - 0s - loss: 1.6874 - acc: 0.3177
Epoch 285/1000
1539/1539 - 0s - loss: 1.6876 - acc: 0.3086
Epoch 286/1000
1539/1539 - 0s - loss: 1.6871 - acc: 0.3106
Epoch 287/1000
1539/1539 - 0s - loss: 1.6876 - acc: 0.3132
Epoch 288/1000
1539/1539 - 0s - loss: 1.6882 - acc: 0.3067
Epoch 289/1000
1539/1539 - 0s - loss: 1.6867 - acc: 0.3106
Epoch 290/1000
1539/1539 - 0s - loss: 1.6879 - acc: 0.3112
Epoch 291/1000
1539/1539 - 0s - loss: 1.6877 - acc: 0.3112
Epoch 292/1000
1539/1539 - 0s - loss: 1.6873 - acc: 0.3099
Epoch 293/1000
1539/1539 - 0s - loss: 1.6874 - acc: 0.3112
Epoch 294/1000
1539/1539 - 0s - loss: 1.6869 - acc: 0.3119
Epoch 295/1000
1539/1539 - 0s - loss: 1.6862 - acc: 0.3158
Epoch 296/1000
1539/1539 - 0s - loss: 1.6869 - acc: 0.3138
Epoch 297/1000
1539/1539 - 0s - loss: 1.6869 - acc: 0.31

Epoch 420/1000
1539/1539 - 0s - loss: 1.6781 - acc: 0.3190
Epoch 421/1000
1539/1539 - 0s - loss: 1.6784 - acc: 0.3177
Epoch 422/1000
1539/1539 - 0s - loss: 1.6771 - acc: 0.3164
Epoch 423/1000
1539/1539 - 0s - loss: 1.6791 - acc: 0.3184
Epoch 424/1000
1539/1539 - 0s - loss: 1.6782 - acc: 0.3171
Epoch 425/1000
1539/1539 - 0s - loss: 1.6766 - acc: 0.3203
Epoch 426/1000
1539/1539 - 0s - loss: 1.6772 - acc: 0.3164
Epoch 427/1000
1539/1539 - 0s - loss: 1.6768 - acc: 0.3197
Epoch 428/1000
1539/1539 - 0s - loss: 1.6779 - acc: 0.3229
Epoch 429/1000
1539/1539 - 0s - loss: 1.6765 - acc: 0.3184
Epoch 430/1000
1539/1539 - 0s - loss: 1.6776 - acc: 0.3164
Epoch 431/1000
1539/1539 - 0s - loss: 1.6768 - acc: 0.3210
Epoch 432/1000
1539/1539 - 0s - loss: 1.6764 - acc: 0.3171
Epoch 433/1000
1539/1539 - 0s - loss: 1.6773 - acc: 0.3203
Epoch 434/1000
1539/1539 - 0s - loss: 1.6774 - acc: 0.3210
Epoch 435/1000
1539/1539 - 0s - loss: 1.6767 - acc: 0.3184
Epoch 436/1000
1539/1539 - 0s - loss: 1.6765 - acc: 0.32

Epoch 559/1000
1539/1539 - 0s - loss: 1.6689 - acc: 0.3210
Epoch 560/1000
1539/1539 - 0s - loss: 1.6699 - acc: 0.3203
Epoch 561/1000
1539/1539 - 0s - loss: 1.6698 - acc: 0.3216
Epoch 562/1000
1539/1539 - 0s - loss: 1.6687 - acc: 0.3177
Epoch 563/1000
1539/1539 - 0s - loss: 1.6689 - acc: 0.3190
Epoch 564/1000
1539/1539 - 0s - loss: 1.6697 - acc: 0.3203
Epoch 565/1000
1539/1539 - 0s - loss: 1.6682 - acc: 0.3223
Epoch 566/1000
1539/1539 - 0s - loss: 1.6680 - acc: 0.3229
Epoch 567/1000
1539/1539 - 0s - loss: 1.6684 - acc: 0.3236
Epoch 568/1000
1539/1539 - 0s - loss: 1.6687 - acc: 0.3229
Epoch 569/1000
1539/1539 - 0s - loss: 1.6689 - acc: 0.3255
Epoch 570/1000
1539/1539 - 0s - loss: 1.6687 - acc: 0.3229
Epoch 571/1000
1539/1539 - 0s - loss: 1.6680 - acc: 0.3190
Epoch 572/1000
1539/1539 - 0s - loss: 1.6690 - acc: 0.3203
Epoch 573/1000
1539/1539 - 0s - loss: 1.6696 - acc: 0.3177
Epoch 574/1000
1539/1539 - 0s - loss: 1.6685 - acc: 0.3262
Epoch 575/1000
1539/1539 - 0s - loss: 1.6680 - acc: 0.32

Epoch 698/1000
1539/1539 - 0s - loss: 1.6630 - acc: 0.3255
Epoch 699/1000
1539/1539 - 0s - loss: 1.6643 - acc: 0.3249
Epoch 700/1000
1539/1539 - 0s - loss: 1.6628 - acc: 0.3242
Epoch 701/1000
1539/1539 - 0s - loss: 1.6630 - acc: 0.3275
Epoch 702/1000
1539/1539 - 0s - loss: 1.6626 - acc: 0.3249
Epoch 703/1000
1539/1539 - 0s - loss: 1.6628 - acc: 0.3255
Epoch 704/1000
1539/1539 - 0s - loss: 1.6644 - acc: 0.3281
Epoch 705/1000
1539/1539 - 0s - loss: 1.6633 - acc: 0.3307
Epoch 706/1000
1539/1539 - 0s - loss: 1.6630 - acc: 0.3223
Epoch 707/1000
1539/1539 - 0s - loss: 1.6627 - acc: 0.3229
Epoch 708/1000
1539/1539 - 0s - loss: 1.6645 - acc: 0.3268
Epoch 709/1000
1539/1539 - 0s - loss: 1.6626 - acc: 0.3307
Epoch 710/1000
1539/1539 - 0s - loss: 1.6623 - acc: 0.3262
Epoch 711/1000
1539/1539 - 0s - loss: 1.6625 - acc: 0.3216
Epoch 712/1000
1539/1539 - 0s - loss: 1.6634 - acc: 0.3275
Epoch 713/1000
1539/1539 - 0s - loss: 1.6621 - acc: 0.3314
Epoch 714/1000
1539/1539 - 0s - loss: 1.6627 - acc: 0.32

Epoch 837/1000
1539/1539 - 0s - loss: 1.6596 - acc: 0.3255
Epoch 838/1000
1539/1539 - 0s - loss: 1.6612 - acc: 0.3288
Epoch 839/1000
1539/1539 - 0s - loss: 1.6603 - acc: 0.3249
Epoch 840/1000
1539/1539 - 0s - loss: 1.6599 - acc: 0.3255
Epoch 841/1000
1539/1539 - 0s - loss: 1.6601 - acc: 0.3275
Epoch 842/1000
1539/1539 - 0s - loss: 1.6609 - acc: 0.3229
Epoch 843/1000
1539/1539 - 0s - loss: 1.6589 - acc: 0.3294
Epoch 844/1000
1539/1539 - 0s - loss: 1.6599 - acc: 0.3275
Epoch 845/1000
1539/1539 - 0s - loss: 1.6589 - acc: 0.3268
Epoch 846/1000
1539/1539 - 0s - loss: 1.6609 - acc: 0.3262
Epoch 847/1000
1539/1539 - 0s - loss: 1.6600 - acc: 0.3223
Epoch 848/1000
1539/1539 - 0s - loss: 1.6620 - acc: 0.3249
Epoch 849/1000
1539/1539 - 0s - loss: 1.6594 - acc: 0.3242
Epoch 850/1000
1539/1539 - 0s - loss: 1.6595 - acc: 0.3307
Epoch 851/1000
1539/1539 - 0s - loss: 1.6602 - acc: 0.3262
Epoch 852/1000
1539/1539 - 0s - loss: 1.6596 - acc: 0.3255
Epoch 853/1000
1539/1539 - 0s - loss: 1.6599 - acc: 0.32

Epoch 976/1000
1539/1539 - 0s - loss: 1.6565 - acc: 0.3294
Epoch 977/1000
1539/1539 - 0s - loss: 1.6577 - acc: 0.3307
Epoch 978/1000
1539/1539 - 0s - loss: 1.6578 - acc: 0.3301
Epoch 979/1000
1539/1539 - 0s - loss: 1.6569 - acc: 0.3288
Epoch 980/1000
1539/1539 - 0s - loss: 1.6572 - acc: 0.3320
Epoch 981/1000
1539/1539 - 0s - loss: 1.6571 - acc: 0.3275
Epoch 982/1000
1539/1539 - 0s - loss: 1.6570 - acc: 0.3281
Epoch 983/1000
1539/1539 - 0s - loss: 1.6582 - acc: 0.3281
Epoch 984/1000
1539/1539 - 0s - loss: 1.6567 - acc: 0.3223
Epoch 985/1000
1539/1539 - 0s - loss: 1.6583 - acc: 0.3216
Epoch 986/1000
1539/1539 - 0s - loss: 1.6569 - acc: 0.3340
Epoch 987/1000
1539/1539 - 0s - loss: 1.6571 - acc: 0.3262
Epoch 988/1000
1539/1539 - 0s - loss: 1.6578 - acc: 0.3262
Epoch 989/1000
1539/1539 - 0s - loss: 1.6566 - acc: 0.3281
Epoch 990/1000
1539/1539 - 0s - loss: 1.6576 - acc: 0.3262
Epoch 991/1000
1539/1539 - 0s - loss: 1.6562 - acc: 0.3262
Epoch 992/1000
1539/1539 - 0s - loss: 1.6565 - acc: 0.32

# Save the Trained Model

In [16]:
#Save the mod#el
model.save("z3_Nueral_network_model.h5")

# Quantifying the Model
Testing data to validate our model. Determine the validity of model (i.e. the ability to predict new and previously unseen data points)

In [17]:
# Evaluate the model using the testing data
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

513/513 - 0s - loss: 1.8079 - acc: 0.2339
Loss: 1.8079279228957772, Accuracy: 0.23391813039779663


## Making Predictions with new data

Use trained model to make predictions using `model.predict`

## Making Predictions with new data
<b>Label asignments</b>


<b>Test Data was taken from the first row:</b>


new_data2


In [18]:
import numpy as np
new_data =   np.array([[2012,12, 49, 7, 23, 0, 281.34, 70, 1016, 2]])
new_data_2 = np.array([[2017,7, 28, 6, 13, 27, 292.06, 88, 1017, 1]]) 

In [19]:
# Test 1
print(f"Predicted class: {model.predict_classes(new_data)}")

ValueError: Error when checking input: expected dense_input to have shape (11,) but got array with shape (10,)

In [ ]:
# Test 2
print(f"Predicted class: {model.predict_classes(new_data)}")

# Evaluate the Model

In [ ]:
# Load the model
#from tensorflow.keras.models import load_model
#from tensorflow.python.ops.init_ops import Zeros

#model = load_model("z_Nueral_network_model.h5")